# Alibi
---
Expliration of the library


### Model Explanations
|Method|Models|Explanations|Classification|Regression|Tabular|Text|Images|Categorical features|Train set required|Distributed|
|:---|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---|:---:|
|[ALE](https://docs.seldon.io/projects/alibi/en/latest/methods/ALE.html)|BB|global|✔|✔|✔| | | |✔| |
|[Anchors](https://docs.seldon.io/projects/alibi/en/latest/methods/Anchors.html)|BB|local|✔| |✔|✔|✔|✔|For Tabular| |
|[CEM](https://docs.seldon.io/projects/alibi/en/latest/methods/CEM.html)|BB* TF/Keras|local|✔| |✔| |✔| |Optional| |
|[Counterfactuals](https://docs.seldon.io/projects/alibi/en/latest/methods/CF.html)|BB* TF/Keras|local|✔| |✔| |✔| |No| |
|[Prototype Counterfactuals](https://docs.seldon.io/projects/alibi/en/latest/methods/CFProto.html)|BB* TF/Keras|local|✔| |✔| |✔|✔|Optional| |
|[Integrated Gradients](https://docs.seldon.io/projects/alibi/en/latest/methods/IntegratedGradients.html)|TF/Keras|local|✔|✔|✔|✔|✔|✔|Optional| |
|[Kernel SHAP](https://docs.seldon.io/projects/alibi/en/latest/methods/KernelSHAP.html)|BB|local <br></br>global|✔|✔|✔| | |✔|✔|✔|
|[Tree SHAP](https://docs.seldon.io/projects/alibi/en/latest/methods/TreeSHAP.html)|WB|local <br></br>global|✔|✔|✔| | |✔|Optional| | 




In [ ]:
!pip install alibi[shap]

     |████████████████████████████████| 317kB 7.9MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 394.5MB 33kB/s 
     |████████████████████████████████| 358kB 47.5MB/s 
     |████████████████████████████████| 28.0MB 86kB/s 
     |████████████████████████████████| 471kB 42.3MB/s 
     |████████████████████████████████| 2.9MB 30.9MB/s 
     |████████████████████████████████| 3.8MB 24.6MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491630 sha256=d1689a0c4a9a7e1f5f6d3869e79c31ace857cf3760ca7e4775c6f28336aac4bf
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052158 sha256=26242102658ca952ada44f82dbfe2bbce0e2195f1d9c1717a2b7af55747ec64d
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb6235c56d014d224bca760d15d7cbdd820813085ffcd35d
Su

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import seaborn as sns
from google.colab import drive
import joblib
import matplotlib as plt

from sklearn.metrics import classification_report, plot_confusion_matrix, plot_roc_curve, accuracy_score, confusion_matrix

import alibi

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
path_mv = "drive/MyDrive/01-Education/03-PhD/2021/Courses/HCI/hci_code/"
path_mg = "drive/MyDrive/HCI/hci_code/"
path_nj = ""

# change to your private path
gdrive_project_root = path_mv 

# Load data and replicate the model

In [ ]:
X = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/X_3cls.csv')
y = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/y_3cls.csv')
X_train = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/X_train_3cls.csv')
X_test = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/X_test_3cls.csv')
y_train = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/y_train_3cls.csv')
y_test = pd.read_csv(gdrive_project_root + 'data/processed/3_cls_model_input/y_test_3cls.csv')

In [ ]:
clf = joblib.load(gdrive_project_root+'models/CLF_3classes_GBM-PHC-AASTR_RandomForest_330estimators_42_random_state.pkl')

In [ ]:
predictions = clf.predict(X_test)
print("Model accuracy: %s \n" % accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
plot_confusion_matrix(clf, X_test, y_test,cmap=plt.cm.Blues, display_labels=["GBM", "PHC", "AASTR"], normalize = "true")

# Accumulated local effects

In [ ]:
from alibi.explainers.ale import ALE, plot_ale
target_names = ["GBM", "PHC", "AASTR"]

The feature effects cannot be read off immediately because the prediction function includes the effects of all features. What we need is a procedure to block out the effects of all other features to uncover the true effect of RM only. This is exactly what the ALE approach does by averaging the differences of predictions across small intervals of the feature.

https://docs.seldon.io/projects/alibi/en/latest/examples/ale_regression_boston.html

https://docs.seldon.io/projects/alibi/en/latest/examples/ale_classification.html



In [ ]:
# logit_fun_lr = clf.decision_function # no decision function in random forest model.. it's probabilistic
proba_fun_lr = clf.predict_proba 

In [ ]:
proba_ale_lr = ALE(proba_fun_lr, feature_names=list(X.columns), target_names=target_names)

proba_exp_lr = proba_ale_lr.explain(X_train.values)


In [ ]:
plot_ale(proba_exp_lr, n_cols=2, fig_kw={'figwidth': 15, 'figheight': 15});


Note that, in this case, the ALE are in the units of relative probability mass, i.e. given a feature value how much more (less) probability does the model assign to each class relative to the mean prediction. This also means that any increase in relative probability of one class must result into a decrease in probability of another class. In fact, the ALE curves summed across classes result in 0 as a direct consequence of conservation of probability:

# Anchors
---
https://docs.seldon.io/projects/alibi/en/latest/methods/Anchors.html


In [ ]:
target_names

In [ ]:
predict_fn = lambda x: clf.predict(x)
explainer = AnchorTabular(predict_fn, list(X.columns))

In [ ]:
explainer.fit(X_train.values, disc_perc=(25, 50, 75))


In [ ]:
idx = 4
print('Prediction: ', target_names[explainer.predictor(X_test.values[idx].reshape(1, -1))[0]])

In [ ]:
for index in range(len(X_test)):
  print('--------------%d--------------------' % index)
  explanation = explainer.explain(X_test.values[index], threshold=0.95)
  print('Prediction: %s' % target_names[explainer.predictor(X_test.values[index].reshape(1, -1))[0] - 1])
  print('Anchor: %s' % (' AND '.join(explanation.anchor)))
  print('Precision: %.2f' % explanation.precision)
  print('Coverage: %.2f' % explanation.coverage)

# Contrastive Explanations Method (CEM)
---
The Contrastive Explanation Method (CEM) can generate black box model explanations in terms of pertinent positives (PP) and pertinent negatives (PN). For PP, it finds what should be minimally and sufficiently present (e.g. important pixels in an image) to justify its classification. PN on the other hand identify what should be minimally and necessarily absent from the explained instance in order to maintain the original prediction.




## No implemented visualizations!

In [ ]:
from alibi.explainers import CEM

In [ ]:
XX = X_test.values[idx].reshape((1,) + X_test.values[idx].shape)
print('Prediction on instance to be explained: {}'.format(target_names[np.argmax(clf.predict_proba(XX))]))
print('Prediction probabilities for each class on the instance: {}'.format(clf.predict_proba(XX)))

In [ ]:
mode = 'PN'  # 'PN' (pertinent negative) or 'PP' (pertinent positive)
shape = (1,) + X_train.values.shape[1:]  # instance shape
kappa = .2  # minimum difference needed between the prediction probability for the perturbed instance on the
            # class predicted by the original instance and the max probability on the other classes
            # in order for the first loss term to be minimized
beta = .1  # weight of the L1 loss term
c_init = 10.  # initial weight c of the loss term encouraging to predict a different class (PN) or
              # the same class (PP) for the perturbed instance compared to the original instance to be explained
c_steps = 10  # nb of updates for c
max_iterations = 1000  # nb of iterations per value of c
feature_range = (X_train.values.min(axis=0).reshape(shape)-.1,  # feature range for the perturbed instance
                 X_train.values.max(axis=0).reshape(shape)+.1)  # can be either a float or array of shape (1xfeatures)
# clip = (-1001.,1001.)  # gradient clipping
clf_init = 1e-2  # initial learning rate

In [ ]:
# define model
predict_fn = lambda x: clf.predict(x)

# initialize CEM explainer and explain instance
cem = CEM(predict_fn, mode, shape, kappa=kappa, beta=beta, feature_range=feature_range,
          max_iterations=max_iterations, c_init=c_init, c_steps=c_steps,
          learning_rate_init=clf_init, clip=clip)
cem.fit(x_train, no_info_type='median')  # we need to define what feature values contain the least
                                         # info wrt predictions
                                         # here we will naively assume that the feature-wise median
                                         # contains no info; domain knowledge helps!
explanation = cem.explain(XX, verbose=False)

# Counterfactuals 
---
This method is based on the Interpretable Counterfactual Explanations Guided by Prototypes paper which proposes a fast, model agnostic method to find interpretable counterfactual explanations for classifier predictions by using class prototypes.


https://docs.seldon.io/projects/alibi/en/latest/examples/cfproto_housing.html

## No implemented visualizations


# Tree SHAP
---


In [ ]:
import xgboost as xgb
from scipy.special import expit
invlogit=expit

In [ ]:
def plot_conf_matrix(y_test, y_pred, class_names):
    """
    Plots confusion matrix. Taken from:
    http://queirozf.com/entries/visualizing-machine-learning-models-examples-with-scikit-learn-and-matplotlib
    """

    matrix = confusion_matrix(y_test,y_pred)


    # place labels at the top
    plt.gca().xaxis.tick_top()
    plt.gca().xaxis.set_label_position('top')

    # plot the matrix per se
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)

    # plot colorbar to the right
    plt.colorbar()

    fmt = 'd'

    # write the number of predictions in each bucket
    thresh = matrix.max() / 2.
    for i, j in product(range(matrix.shape[0]), range(matrix.shape[1])):

        # if background is dark, use a white number, and vice-versa
        plt.text(j, i, format(matrix[i, j], fmt),
             horizontalalignment="center",
             color="white" if matrix[i, j] > thresh else "black")

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.tight_layout()
    plt.ylabel('True label',size=14)
    plt.xlabel('Predicted label',size=14)
    plt.show()

def predict(xgb_model, dataset, proba=False, threshold=0.5):
    """
    Predicts labels given a xgboost model that outputs raw logits.
    """

    y_pred = model.predict(dataset)  # raw logits are predicted
    y_pred_proba = invlogit(y_pred)
    if proba:
        return y_pred_proba
    y_pred_class = np.zeros_like(y_pred)
    y_pred_class[y_pred_proba >= threshold] = 1  # assign a label

    return y_pred_class

In [ ]:
def fit( XXX, yyy, X_valid, y_valid):
        print('XGBoost, train data shape        {}'.format(X.shape))
        print('XGBoost, validation data shape   {}'.format(X_valid.shape))
        print('XGBoost, train labels shape      {}'.format(y.shape))
        print('XGBoost, validation labels shape {}'.format(y_valid.shape))
        param = {'max_depth': 2, 'eta': 1, 'silent': 1,
                             'objective': 'multi:softmax', 'num_class': 3}

        train = xgb.DMatrix(data=XXX,
                            label=yyy, feature_names=list(X.columns))
        valid = xgb.DMatrix(data=X_valid,
                            label=y_valid, feature_names=list(X.columns))
        estimator = xgb.train(param, dtrain=train,
                                   evals=[(train, 'train'), (valid, 'valid')],
                                   )
        return estimator

In [ ]:
model = fit(X_train.values, y_train.values - 1 , X_test.values, y_test.values-1)



In [ ]:
y_pred_train = predict(model, xgb.DMatrix(
    np.ascontiguousarray(X_train),
    label=np.ascontiguousarray(y_train),
    feature_names=list(X.columns),
))
y_pred_test = predict(model, xgb.DMatrix(
    np.ascontiguousarray(X_test),
    label=np.ascontiguousarray(y_test),
    feature_names=list(X.columns),
))

In [ ]:
print(f'Train accuracy:  {round(100*accuracy_score(y_train, y_pred_train), 4)}  %.')
print(f'Test  accuracy:  {round(100*accuracy_score(y_test, y_pred_test), 4)}%.')

In [ ]:
from functools import partial


def _get_importance(model, measure='weight'):
    """
    Retrieves the feature importances from an xgboost
    models, measured according to the criterion `measure`.
    """

    imps = model.feature_importances_
    names, vals = list(imps.keys()), list(imps.values())
    sorter = np.argsort(vals)
    s_names, s_vals = tuple(zip(*[(names[i], vals[i]) for i in sorter]))

    return s_vals[::-1], s_names[::-1]

def plot_importance(feat_imp, feat_names, ax=None, **kwargs):
    """
    Create a horizontal barchart of feature effects, sorted by their magnitude.
    """

    left_x, step ,right_x = kwargs.get("left_x", 0), kwargs.get("step", 50), kwargs.get("right_x")
    xticks = np.arange(left_x, right_x, step)
    xlabel = kwargs.get("xlabel", 'Feature effects')
    xposfactor = kwargs.get("xposfactor", 1)
    textfont = kwargs.get("text_fontsize", 25) # 16
    yticks_fontsize = kwargs.get("yticks_fontsize", 25)
    xlabel_fontsize = kwargs.get("xlabel_fontsize", 30)
    textxpos = kwargs.get("textxpos", 60)
    textcolor = kwargs.get("textcolor", 'white')

    if ax:
        fig = None
    else:
        fig, ax = plt.subplots(figsize=(10, 5))

    y_pos = np.arange(len(feat_imp))
    ax.barh(y_pos, feat_imp)

    ax.set_yticks(y_pos)
    ax.set_yticklabels(feat_names, fontsize=yticks_fontsize)
    ax.set_xticklabels(xticks, fontsize=30, rotation=45)
    ax.invert_yaxis()                  # labels read top-to-bottom
    ax.set_xlabel(xlabel, fontsize=xlabel_fontsize)
    ax.set_xlim(left=left_x, right=right_x)

    for i, v in enumerate(feat_imp):
#         if v<0:
        textxpos = xposfactor*textxpos
        ax.text(v - textxpos, i + .25, str(round(v, 3)), fontsize=textfont, color=textcolor)
    return ax, fig

get_importance = partial(_get_importance, clf)

In [ ]:
imp_by_weight_v, imp_by_weight_n = get_importance()
imp_by_gain_v, imp_by_gain_n = get_importance(measure='total_gain')
imp_by_a_gain_v, imp_by_a_gain_n = get_importance(measure='gain')